# Machine Learning Systems Assignment 2

<a target="_blank" href="https://colab.research.google.com/github/mlsyscourse/assignment2/blob/main/mlsys_hw2.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Assignment due: Feb 26, 2025, 11:59 pm, Eastern Time**.

Tensor program optimization is a very important part in machine learning compilation.
It enables operators in machine learning (e.g., matrix multiplication, convolution, reduction, etc.)
to run efficiently on various modern hardware.
In this assignment, you will use a tensor program optimizing interface in [TVM](https://tvm.apache.org)
to schedule and automatically tune typical operators.

* **Be prepared** -- in this assignment you may need to learn many new terminologies and concepts.
* You should work on this assignment **individually** -- it is not a team assignment.
* This assignment **requires** NVIDIA GPU with CUDA environment. You can do the assignment on Google Colab (by clicking the badge above) or any other GPU server that you have access to. **Important note: we will test your submission with an NVIDIA T4 GPU, which is free available on Google Colab. So if you do the assignment on other environments, we recommend you to test it on Colab before submission.**
* This assignment is pure Python. No C++ is needed in this assignment.
* Please check out the end of this notebook for the assignment submission requirement.
* Please do not share your solution on publicly available websites (e.g., GitHub).
* **About testing and grading.** The assignment will be automatically graded. You can submit multiple times, and the time stamp of that submission will be used in determining any late penalties. We also encourage you to create your own test cases, which helps you confirm the correctness of your code.


## Preparation

### Using Google Colab

If you are using Google Colab environment, please make a copy of this notebook file by selecting "Save a copy in Drive" from the "File" menu, and then run the code block below to set up workspace. After cloning, you will see the cloned repository in the "Files" bar on the left.

In [3]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p 15442
%cd /content/drive/MyDrive/15442
!git clone https://github.com/mlsyscourse/assignment2.git
%cd /content/drive/MyDrive/15442/assignment2

Mounted at /content/drive
/content/drive/MyDrive
/content/drive/MyDrive/15442
fatal: destination path 'assignment2' already exists and is not an empty directory.
/content/drive/MyDrive/15442/assignment2


Then run the following cell to install TVM dependency.

**Note.** The PyPI package installed below only has CPU support.
It is fine to use this CPU package when you implement the
first five tasks of part 1.
We will have instructions on how to install the GPU package
later on.

In [7]:
!python3 -m pip install mlc-ai-cpu -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels


We can validate that the dependency is successfully installed by running the following.

In [8]:
import tvm
tvm.__path__

['/usr/local/lib/python3.11/dist-packages/tvm']

### Using local environment

If you are using local/server environment (requiring NVIDIA GPU and CUDA ≥ 11.7), please clone this repository.

```shell
git clone https://github.com/mlsyscourse/assignment2.git
cd assignment2
export PYTHONPATH=$PWD:$PYTHONPATH
```

We recommend you to create a conda environment and install the dependency package in the environment.
```shell
conda create --name 15442 python=3.11
conda activate 15442
python3 -m pip install --force-reinstall mlc-ai-cu123 -f https://mlc.ai/wheels
```

Please also select the Python interpreter in this conda environment and use it to run this notebook.
We can validate that the dependency is successfully installed by running the following.

In [ ]:
import tvm
tvm.__path__

['/path/to/conda/envs/15442/lib/python3.11/site-packages/tvm']


## Part 1. Optimize Matrix Multiplication (80 pt)

In part 1, you will manually optimize a fused "GeMM + ReLU + add" operator with TVM and run it
on the NVIDIA GPU in your environment.
The "GeMM + ReLU + add" pattern is common in machine learning models when activation
and residual-add operations are involved.
Formally, for given 2-dim tensors $A$, $B$, and $C$, we want to compute the following $D$:

$$
D = \mathrm{ReLU} (A B) + C.
$$

We define this operator using [TensorIR](https://arxiv.org/pdf/2207.04296.pdf),
the tensor program level intermediate representation (IR) in TVM, as follows.
You can also find this definition in `gemm_relu_add.py`.

```python
from tvm import tir
from tvm.script import tir as T

M = 2048
N = 2048
K = 2048


@T.prim_func
def gemm_relu_add(
    A: T.Buffer((M, K), "float32"),
    B: T.Buffer((K, N), "float32"),
    C: T.Buffer((M, N), "float32"),
    D: T.Buffer((M, N), "float32"),
) -> None:
    matmul = T.alloc_buffer((M, N), "float32", scope="global")
    relu = T.alloc_buffer((M, N), "float32", scope="global")
    # Compute GeMM
    for i, j, k in T.grid(M, N, K):
        with T.block("gemm"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            vk = T.axis.reduce(K, k)
            with T.init():
                matmul[vi, vj] = T.float32(0)
            matmul[vi, vj] += A[vi, vk] * B[vk, vj]
    # Compute ReLU
    for i, j in T.grid(M, N):
        with T.block("relu"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            relu[vi, vj] = T.max(matmul[vi, vj], T.float32(0))
    # Compute add
    for i, j in T.grid(M, N):
        with T.block("add"):
            vi = T.axis.spatial(M, i)
            vj = T.axis.spatial(N, j)
            D[vi, vj] = relu[vi, vj] + C[vi, vj]


sch = tir.Schedule(gemm_relu_add)
```

Let's go through this piece of code and look into its components.

* The function takes the four tensors `A`, `B`, `C` and `D` as inputs with the corresponding shape and dtype.
You may notice that the tensor `D`, which we want to compute, is passed in as an argument, rather than
a function return value.
This is what we call the "destination-passing style," where we allocate the destination tensor
before calling this function, and pass in the destination tensor as an argument.
Destination-passing style is widely adopted by low-level operator libraries (e.g.,
[cuBLAS](https://docs.nvidia.com/cuda/cublas/index.html)) and the tensor program level
abstractions in machine learning compilers.
* The first two lines of the function body define the tensors where intermediate results from
GeMM and ReLU are stores. Their memory scope is global at this moment.
* Next, there are the three sequential for-loop blocks defining the computation.
  * The first one defines the GeMM computation. For a given `vi` and `vj`, the loop body
  reads from `A` and `B`, sums up the values along `vk`, and stores the summation result into
  the intermediate tensor `matmul`, with `T.float32(0)` being the initial value of the summation.
  * The second one defines the ReLU computation, which element-wisely takes the maximum
  of `matmul` and zero.
  * The third one describes the element-wise addition of the ReLU result and the input tensor `C`.
* After defining the function, we create a `tir.Schedule` instance with regard to this function.
`tir.Schedule` is the core tool in this assignment you will use to optimize this function.
`tir.Schedule` provides the a set of function transformations (which we call "schedule primitives")
that help accelerate the function execution on hardware.
We will introduce these schedule primitives later.

Your goal in this part is to optimize this GeMM + ReLU + add workload so that it can efficiently run on GPU.
Usually, optimizing such a workload of matrix multiplication with epilogues (e.g., ReLU and add here)
for GPU, five major steps are required:

1. shared memory tiling,
2. register tiling,
3. cooperative fetching,
4. write cache.
5. epilogue fusion.

Before you start, here is a note on grading.
When grading your submission in this part, we evaluate your optimization from two aspects.
* The first and most basic one is the **numerical correctness**.
This being said, after optimizing the workload, your function should produce the
same numerical results as the original workload.
More concretely, in this assignment, it means that the function after your optimization
should still compute GeMM + ReLU + addition.
We provide the tests for you to check the numerical correctness of your optimization.
But you may not be able to run the test until you have implemented all optimizations.
* The other aspect is whether you properly implemented the optimizations.
Unfortunately, it is hard to automatically check this,
and **there is no test** for you to test it in Python.
Your submission will be **manually** checked to see if each task is properly implemented.
On the other hand, we provide the optimized function after **finishing all five tasks** in `reference.py` for your reference.
So you can see what a final function may look like,
and you can also refer to it after you implementing each task.
*Please note that you do not have to get the exactly same function as the reference one.*


Now, let's get started.

### Task 1. Shared memory tiling (20 pt)

Our optimization is centered around the GeMM part.
This is because though we have three stages (GeMM, ReLU and add) in this workload,
GeMM has the the heaviest computation and the most memory accesses.
Once we have optimized the GeMM, the rest parts can be optimized accordingly,
based on how the GeMM is optimized.

You have learned about the concept of **thread blocks** in GPU architectures from the lectures.
To optimize a GeMM workload, people usually partition the destination matrix (`matmul` for our case)
into multiple tiles along both the row and column dimensions, and assign one thread block
for the computation of each tile.

For example, in the figure below, a thread block computes a `(tile_x, tile_y)` tile.
That is to say, this particular thread block will iterate over a `(tile_x, K)` region of `A`
and a `(K, tile_y)` region of `B`, compute the matrix multiplication of these two regions,
and write the results into the `(tile_x, tile_y)` tile.
We can use Python notation to formally describe the tiling:
for a given thread block, assuming the tile that this thread block computes
starts from `offset_x` on the `M` dimension and `offset_y` on the `N` dimension,
then the thread block computes the following:
```python
matmul[offset_x : offset_x + tile_x, offset_y : offset_y + tile_y] = (
    A[offset_x : offset_x + tile_x, :] @ B[:, offset_y : offset_y + tile_y]
)
```

<img src="https://raw.githubusercontent.com/mlsyscourse/assignment2/main/figure/shared-memory-tiling.jpg" alt="figure/shared-memory-tiling.jpg" width="100%">

The next step is about reducing the memory access pressure of the GeMM operator.
This is done via leveraging **shared memory** available in GPU, which you also learned about from lectures:
rather than directly reading from `A` and `B` (which reside in global memory)
at the time of computing multiplication, each thread block first loads the `A` region
and `B` region it needs from global memory to shared memory.

Just like CPU cache, while shared memory can offer much faster access speed,
it has relatively smaller size than global memory.
Usually, the size of shared memory available in a thread block is no more than 48KB.
In our case, given each float32 element has four bytes,
the `A` region and `B` region that a thread block needs
have size `(tile_x * K * 4) + (tile_y * K * 4)` bytes.
which is far more than the available shared memory size a thread block can use.
To address this issue, you also need split the long stripe of `A` and `B` into
multiple chunks along the `K` dimension.
By doing this, now you are able to go through `A` chunks and `B` chunks one at a time simultaneously,
and load a single `A` tile and `B` chunk into shared memory, which fits the available
shared memory size.
This can be formally written as

```python
matmul[offset_x : offset_x + tile_x, offset_y : offset_y + tile_y] = sum(
    A[offset_x : offset_x + tile_x, k : k + tile_k]
    @ B[k : k + tile_k, offset_y : offset_y + tile_y]
    for k in range(0, n, tile_k)
)
```


We can analyze how much global memory accesses we reduce by leveraging shared memory:

* There are $M \times K$ elements in $A$ and $N \times K$ elements in $B$.
When not using shared memory, each element of $A$ is read for $N$ times,
and each element of $B$ is read for $M$ times.
So number of accesses to $A$ and $B$ in global memory are both $MNK$.
* With shared memory tiling, the number of times each element of $A$ is read becomes $\frac{N}{\mathrm{tile}_y}$,
and the number of times each element of $B$ is read becomes $\frac{M}{\mathrm{tile}_x}$.
And the total number of accesses to $A$ and $B$ are divided by $\mathrm{tile}_y$ and $\mathrm{tile}_x$ respectively.


You need to implement the shared memory tiling in the `shared_memory_tiling` function in `gemm_relu_add.py`.
Below are the schedule primitives you will use in this task.
We provide [another Jupyter notebook](https://github.com/mlsyscourse/assignment2/blob/main/schedule_example.ipynb) that walks you through an example
of using some of these schedule primitives for your reference.
Meanwhile, you can click the links below to check out their documentations,
which contain a basic example on how to use the primitive.
We also provide instructions in your TODO area in `gemm_relu_add.py`.

- `split` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.split)
  - It splits one loop into multiple loops that collectively iterate the iteration space of the original loop.
  We use `split` for tiling.
- `reorder` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.reorder)
  - It reorders the specified contiguous loops into a new order.
  We apply `reorder` for loops after splitting for tiling.
- `bind` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.bind)
  - It binds a loop to `blockIdx.x/y/z` or `threadIdx.x/y/z` on GPU,
  so that we can specify the area to compute of a thread block or a single thread.
- `cache_read` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.cache_read)
  - It generates a cache stage for the specified region in the specified memory scope.
  We use `cache_read` to generate the read stages from global memory to shared memory.
- `compute_at` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.compute_at)
  - It moves a computation (e.g., the shared memory read stage) to the location under the specified loop.
  We use `compute_at` to move the shared memory read stages to the right location.

You can run `gemm_relu_add.py` to see how the function looks like after your optimization.
Note: you can insert `sch.show()` at any position to print out the function,
so that you can visually see how the function looks like.

In [4]:
!python3 gemm_relu_add.py

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "float32"), B: T.Buffer((2048, 2048), "float32"), C: T.Buffer((2048, 2048), "float32"), D: T.Buffer((2048, 2048), "float32")):
        T.func_attr({"global_symbol": "gemm_relu_add"})
        # with T.block("root"):
        matmul = T.alloc_buffer((2048, 2048))
        relu = T.alloc_buffer((2048, 2048))
        A_shared = T.alloc_buffer((2048, 2048), scope="shared")
        B_shared = T.alloc_buffer((2048, 2048), scope="shared")
        A_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        B_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        for i_0 in T.thread_binding(32, thread="blockIdx.x"):
            for j_0 in T.thread_binding(32, thread="blockIdx.y"):
                for k_0 in range(32):
                    for ax0_ax1_fused_0 in range(16):
                        for ax0_ax1_fused_1 in T.thread_bi

### Task 2. Register tiling (20 pt)

In the previous subsection, we focus on handling the `matmul` region that **a thread block collectively computes**.
In this subsection, we dive deeper to figure out the `matmul` region that **a single thread computes**.

The idea is pretty much similar to the shared memory tiling.
We can also let a single thread compute a tile in `matmul`.
For example, we assign each thread in a thread block to compute a tile of size `(thread_tile_x, thread_tile_y)`.
And moreover, rather than directly load `A` and `B` from shared memory when computing the tile,
we again first load `A` and `B` from shared memory to **local registers**.
Registers are local memory in a running CUDA thread that is running,
and can be accessed in much shorter time than shared memory.
Similarly, we also need to do thread-level tiling along the `K` dimension,
because each thread has very limited number of available registers.
And our goal is to make sure the data that we want to load from `A` and `B` in each thread-level tile can fit into the registers.
With register tiling, we use $(\mathrm{thread\_tile}_x + \mathrm{thread\_tile}_y) \times \mathrm{thread\_tile}_k$
registers for reading data from the shared memory of `A` and `B` in total.


<img src="https://raw.githubusercontent.com/mlsyscourse/assignment2/main/figure/register-tiling.jpg" alt="figure/register-tiling.jpg" width="50%">



It worths noting that, by fixing the thread tile size, we equivalently fixed the number
of threads in each thread block:
a thread block computes a tile of `(tile_x, tile_y)`, and a single thread computes a tile of
`(thread_tile_x, thread_tile_y)`.
So in a thread block, `blockDim.x` is `tile_x / thread_tile_x`, `blockDim.y` is `tile_y / thread_tile_y`,
and the total number of threads in a thread block is `(tile_x / thread_tile_x) * (tile_y / thread_tile_y)`.


You need to implement the register tiling in the `register_tiling` function in `gemm_relu_add.py`.
As mentioned above, your implementation of register tiling should follow the same pattern and steps
as your implementation of shared memory tiling.
So make sure you understand each step of the shared memory tiling before starting this task.
The schedule primitives you will use in this task is the same as task 1.
And please note that for register tiling, you should use `"local"` (which means local registers)
as the scope of `cache_read`, rather than `"shared"` which you used in task 1.

In [12]:
!python3 gemm_relu_add.py

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "float32"), B: T.Buffer((2048, 2048), "float32"), C: T.Buffer((2048, 2048), "float32"), D: T.Buffer((2048, 2048), "float32")):
        T.func_attr({"global_symbol": "gemm_relu_add"})
        # with T.block("root"):
        matmul = T.alloc_buffer((2048, 2048))
        relu = T.alloc_buffer((2048, 2048))
        A_shared = T.alloc_buffer((2048, 2048), scope="shared")
        B_shared = T.alloc_buffer((2048, 2048), scope="shared")
        for i_0 in T.thread_binding(32, thread="blockIdx.x"):
            for j_0 in T.thread_binding(32, thread="blockIdx.y"):
                for k_0 in range(32):
                    for ax0, ax1 in T.grid(64, 64):
                        with T.block("A_shared"):
                            v0 = T.axis.spatial(2048, i_0 * 64 + ax0)
                            v1 = T.axis.spatial(2048, k_0 * 64 + ax1)
        

### Task 3. Cooperative fetching (20 pt)

Throughout the shared memory tiling and register tiling,
we have assigned each thread a `matmul` region to compute.
This concludes our arrangement of the GeMM computation.
Nevertheless, we still have a few more optimization steps till the end.
The first one is **cooperative fetching**, where we optimize the memory load
from global memory to shared memory for both `A` and `B`.

Remember in the "shared memory tiling" subsection, we created stages for
a thread block to read data from `A` and `B` into their respective shared memory.
However, you may notice that how the the data is read remains unspecified.
Specifically, we have known which `A`/`B` region a thread block should read,
but **do not specify which `A`/`B` elements a single thread should read**.
If we do not specify this, by default *every* thread in a thread block will
read the entire `A`/`B` regions that this thread block is responsible for and
write the values into shared memory correspondingly.
Since the shared memory of a thread block is accessible to all threads,
this default behavior, as a result, leads to duplicate global memory reads
and shared memory writes, and fails to reduce the number of global memory accesses.

This motivates us to implement cooperative fetching.
Cooperative fetching means all the threads in a thread block
collectively ("*cooperative*") load values from global memory to shared memory ("*fetching*").
To make sure each element in the tile is read only once (by some thread),
we can divide the number of elements in the shared memory tile evenly by
the number of threads, yielding the number of elements each thread loads.
Then, we let all the threads in the thread block to load values in memory order.

For our example, we can calculate that each thread in a thread block needs to
load `thread_tile_x * thread_tile_y * tile_k / tile_y` elements from `A`,
and likewise, `thread_tile_x * thread_tile_y * tile_k / tile_x` elements from `B`.
If we denote the number of total threads (`(tile_x / thread_tile_x) * (tile_y / thread_tile_y)`) as
`num_threads`, then when loading an `A` tile to shared memory, in the first round all the threads
load the first continuous `num_threads` elements, in the second round they load
the next continuous `num_threads` elements, and keep loading for
`thread_tile_x * thread_tile_y * tile_k / tile_y` times.
This applies to the shared memory load of `B` in the same way.

[The other Jupyter notebook](https://github.com/mlsyscourse/assignment2/blob/main/schedule_example.ipynb) contains a simple example of
cooperative fetching that you might find helpful for understanding,
if you have not went through it.

You need to implement the cooperative fetching in the `cooperative_fetching` function in `gemm_relu_add.py`.
As you can see in the code, since the cooperative fetching for `A` and `B` are the same,
we can write a common schedule function and apply it to the shared memory read stages
of both `A` and `B`.
The new schedule primitive you will use in this task is
`fuse` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.fuse),
which, on the contrary to `split`, fuses multiple loops into a single loop.

In [30]:
!python3 gemm_relu_add.py

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "float32"), B: T.Buffer((2048, 2048), "float32"), C: T.Buffer((2048, 2048), "float32"), D: T.Buffer((2048, 2048), "float32")):
        T.func_attr({"global_symbol": "gemm_relu_add"})
        # with T.block("root"):
        matmul = T.alloc_buffer((2048, 2048))
        relu = T.alloc_buffer((2048, 2048))
        A_shared = T.alloc_buffer((2048, 2048), scope="shared")
        B_shared = T.alloc_buffer((2048, 2048), scope="shared")
        A_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        B_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        matmul_local = T.alloc_buffer((2048, 2048), scope="local")
        for i_0 in T.thread_binding(32, thread="blockIdx.x"):
            for j_0 in T.thread_binding(32, thread="blockIdx.y"):
                for k_0 in range(32):
                    for ax0_ax1_fused_0 in ra

### Task 4. Write Cache (10 pt)

We can now move on to the stage of writing the GeMM computation result into `matmul`.
So far, our GeMM computation still directly writes the results into the global memory of `matmul`,
with each location of `matmul` written for `K` times, as our GeMM takes the summation over the `K` dimension.
This is obviously not optimal.

Instead of writing the computation results directly into global memory,
we can accumulate the summation in local registers,
and write the result into global memory for fewer times after the accumulation is finished.

You need to implement the write cache in the `write_cache` function in `gemm_relu_add.py`.
In this task, you will use the schedule primitive
`cache_write` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.cache_write)
and
`reverse_compute_at` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.reverse_compute_at)
to put the write cache stage at the right location.
Similar to `cache_read` which generates a read cache stage for a computation,
`cache_write` generates a write cache stage for a computation at the specified scope.
`reverse_compute_at` works in basically the same way as `compute_at`,
with the difference being `compute_at` moves a computation **in front of** the target loop into the target loop,
while `reverse_compute_at` moves a computation **behind** the target loop into the target loop.

In [6]:
!python3 gemm_relu_add.py

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "float32"), B: T.Buffer((2048, 2048), "float32"), C: T.Buffer((2048, 2048), "float32"), D: T.Buffer((2048, 2048), "float32")):
        T.func_attr({"global_symbol": "gemm_relu_add"})
        # with T.block("root"):
        A_shared = T.alloc_buffer((2048, 2048), scope="shared")
        B_shared = T.alloc_buffer((2048, 2048), scope="shared")
        A_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        B_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        matmul_local = T.alloc_buffer((2048, 2048), scope="local")
        for i_0 in T.thread_binding(32, thread="blockIdx.x"):
            for j_0 in T.thread_binding(32, thread="blockIdx.y"):
                for k_0 in range(32):
                    for ax0_ax1_fused_0 in range(16):
                        for ax0_ax1_fused_1 in T.thread_binding(16, thread="threa

### Task 5. Epilogue Fusion (10 pt)

The subsections above include the common techniques people use
to optimize a GeMM workload.
Now we can take care of the ReLU and add operators.
Though these two operators are computation-wise much more lightweight
compared to the GeMM,
right now they still account for a quite amount of time,
since they need to read/write directly from/to global memory.
This can be optimized by inlining the lightweight epilogue operators
into the write-back stage of the scheduled GeMM workload,
where the result of GeMM is still in registers.
This optimizes the redundant memory accesses.


You need to implement the epilogue fusion in the `epilogue_fusion` function in `gemm_relu_add.py`,
where you will use `get_block` to retrieve the computation of addition and ReLU,
and use `reverse_compute_inline` [(docs)](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.reverse_compute_inline)
to inline the addition and ReLU into the write-back stage of GeMM.

In [7]:
!python3 gemm_relu_add.py

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((2048, 2048), "float32"), B: T.Buffer((2048, 2048), "float32"), C: T.Buffer((2048, 2048), "float32"), D: T.Buffer((2048, 2048), "float32")):
        T.func_attr({"global_symbol": "gemm_relu_add"})
        # with T.block("root"):
        A_shared = T.alloc_buffer((2048, 2048), scope="shared")
        B_shared = T.alloc_buffer((2048, 2048), scope="shared")
        A_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        B_shared_local = T.alloc_buffer((2048, 2048), scope="local")
        matmul_local = T.alloc_buffer((2048, 2048), scope="local")
        for i_0 in T.thread_binding(32, thread="blockIdx.x"):
            for j_0 in T.thread_binding(32, thread="blockIdx.y"):
                for k_0 in range(32):
                    for ax0_ax1_fused_0 in range(16):
                        for ax0_ax1_fused_1 in T.thread_binding(16, thread="threa

---

### If you are using Google Colab...

Now we need to switch to GPU runtime and install the PyPI package with CUDA support before
running tests.
Please go to "Runtime - Change runtime type" in the navigation bar, and select "T4 GPU".
Then run the cells below to install the PyPI package, verify the installation,
and mount the Google Drive again.

**Note 1.** Be sure to change runtime type to use T4 GPU.
Or otherwise you will see library error when importing TVM in Python below.

**Note 2.** Please disconnect the GPU runtime when you are not using it.
Otherwise you may hit the Colab GPU usage limit and will be temporarily not able to
connect to a GPU in Colab if you are a free Colab user.

In [2]:
!python3 -m pip install mlc-ai-cu123 -f https://mlc.ai/wheels

Looking in links: https://mlc.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 GB 1.2 MB/s eta 0:00:00


In [2]:
import tvm
tvm.__path__

['/usr/local/lib/python3.11/dist-packages/tvm']

In [1]:
# Code to set up the assignment
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/15442/assignment2

Mounted at /content/drive
/content/drive/MyDrive/15442/assignment2


---

Now that you have finished implementing all the optimizations,
you can test the numerical correctness of your code via running the following.

In [8]:
!python3 evaluate.py --test

Passing test round 0...
Passing test round 1...
Passing test round 2...
Passing test round 3...
Passing test round 4...
Passed all tests.


You can evaluate the execution time of function after your optimization via running the following:

In [9]:
!python3 evaluate.py --evaluate-manual

Execution time: 23.88 ms


To help you get a sense of how much performance your optimization gains,
we can evaluate a naive GeMM + ReLU + add function with little optimizations.
Ideally, you should find that your optimization is times faster than the naive optimization.

In [10]:
!python3 evaluate.py --evaluate-naive

Naive function execution time: 280.01 ms


You can print out the CUDA source code of your scheduled workload that TVM generates,
and compare it with the Python script you printed
via `sch.show()`.
As a practice, try to point out which parts in the CUDA source code
are the shared memory, local registers,
the main GeMM computation, cooperative fetching, write cache,
and the fused epilogue respectively.

In [11]:
!python3 evaluate.py --show-cuda


#if (((__CUDACC_VER_MAJOR__ == 11) && (__CUDACC_VER_MINOR__ >= 4)) || \
     (__CUDACC_VER_MAJOR__ > 11))
#define TVM_ENABLE_L2_PREFETCH 1
#else
#define TVM_ENABLE_L2_PREFETCH 0
#endif

#ifdef _WIN32
  using uint = unsigned int;
  using uchar = unsigned char;
  using ushort = unsigned short;
  using int64_t = long long;
  using uint64_t = unsigned long long;
#else
  #define uint unsigned int
  #define uchar unsigned char
  #define ushort unsigned short
  #define int64_t long long
  #define uint64_t unsigned long long
#endif
extern "C" __global__ void __launch_bounds__(256) gemm_relu_add_kernel(float* __restrict__ A, float* __restrict__ B, float* __restrict__ C, float* __restrict__ D);
extern "C" __global__ void __launch_bounds__(256) gemm_relu_add_kernel(float* __restrict__ A, float* __restrict__ B, float* __restrict__ C, float* __restrict__ D) {
  __shared__ float A_shared[4096];
  __shared__ float B_shared[4096];
  float matmul_local[16];
  float A_shared_local[4];
  float B_shared_

## Part 2. Auto Tuning (20 pt)

In this part, you have the chance to step further and apply automatic tuning to
our GeMM + ReLU + add workload with some adaptation to your code in part 1,
so that the optimized workload can run even faster on GPU.

In the previous part, we only implemented one schedule for the workload.
However, there are many other ways to schedule and optimize the workload,
and the particular schedule in part 1 may not be optimal on GPU.
A typical example is the tile size:
you can easily get other schedules by tweaking the
tile sizes for shared memory tiling and register tiling.

To describe the problem we want to tackle with auto tuning --
all the possible choices of schedules form a search space,
and each schedule choice has a corresponding execution time on GPU.
We want to find out the schedule with the shortest execution time.

However, the execution time of a schedule is hard to be predictable,
especially for complicated workloads such as GeMM and convolution
(which we don't cover in this assignment).
This is because the execution time of a scheduled program on GPU
usually depends on multiple factors, making it challenging to build up
analytical model to find the best performant schedule.
This motivates us to build auto tuning tools to help find the
best schedule within the search space on the target GPU.

The problem of auto tuning generally consists of two sub-problems.
The first one is to generate the search space (i.e., describing the
set of programs/schedules in the search space). And the other one
is to find the best performant schedule in the space.

---

Your task in this part is to define the search space for our
GeMM + ReLU + add workload in the function `auto_tuning_schedule` of `auto_tuning.py`.
To make things easier, you can only focus on the different
choices of tile size in this assignment, and ideally your implementation of
the `auto_tuning_schedule` function is adapted from your code in part 1.

Obviously, the most straightforward choice is to directly reuse the schedule in part 1.
Doing this gives us a search space which only contains a single schedule.
You can also manually adjust the tile sizes to check if different tile sizes
offer better performance.

If you want to define a search space that goes beyond a single schedule,
the primitive [`Schedule.sample_perfect_tile`](https://tvm.apache.org/docs/reference/api/python/tir/schedule.html#tvm.tir.schedule.Schedule.sample_perfect_tile)
in `tir.Schedule` helps you to sample the tile sizes from the specified loop.
For example, for a given schedule `sch`, a given loop `i` with loop length `length`,
```python
tile0, tile1, tile2, tile3 = sch.sample_perfect_tile(i, n=4)
```
returns four integer tile sizes `[tile0, tile1, tile2, tile3]` in a list,
such that `tile0 * tile1 * tile2 * tile3 == length`.
And you can thereby use these tile sizes in schedule primitives such as
`split`:
```python
i0, i1, i2, i3 = sch.split(i, factors=[tile0, tile1, tile2, tile3])
```

Now, go ahead and implement function `auto_tuning_schedule`.
We do not provide other examples for you in this part.

**Note:** if you feel unsure about this part and don't know how to start,
just try to copy and paste your part 1 code into this function,
proceed and run the auto tuning code below to see if it is runnable
and gives you an execution time. Then you can come back to update your
`auto_tuning_schedule` implementation.
This may help you build confidence.


---

After defining the search space, let's take a look at the second
sub-problem on how to find the best performance schedule within the search space.

The most basic way of auto tuning is simply iterate over the entire search space,
and evaluate the performance on GPU for every schedule in the space.
As you can imagine, this is effective and straightforward when the
search space is not too large. However, it is time-consuming
to go through every single choice when the search space is large.

Here we use an improved way based on genetic algorithms,
so that we do not have to go through the entire search space.
We will not expand on the details here.
If you are interested, you can refer to the papers of
[AutoTVM](https://proceedings.neurips.cc/paper_files/paper/2018/file/8b5700012be65c9da25f49408d959ca0-Paper.pdf)
and [Meta Schedule](https://proceedings.neurips.cc/paper_files/paper/2022/file/e894eafae43e68b4c8dfdacf742bcbf3-Paper-Conference.pdf)
for more details.

You can run the cell below to automatically tune the workload
with the search space you defined.
**The full auto tuning under the default settings may take at most 5 minutes on Google Colab if you use `sample_perfect_tile`.**

In [4]:
!python3 auto_tuning.py

2025-02-23 22:11:50 [INFO] Logging directory: /tmp/tmpw1lptanv/logs
2025-02-23 22:12:04 [INFO] LocalBuilder: max_workers = 1
2025-02-23 22:12:06 [INFO] LocalRunner: max_workers = 1
2025-02-23 22:12:07 [INFO] [task_scheduler.cc:159] Initializing Task #0: "main"
Traceback (most recent call last):
  File "/content/drive/MyDrive/15442/assignment2/auto_tuning.py", line 150, in <module>
    auto_tune()
  File "/content/drive/MyDrive/15442/assignment2/auto_tuning.py", line 131, in auto_tune
    database = ms.tir_integration.tune_tir(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tvm/meta_schedule/tir_integration.py", line 146, in tune_tir
    return tune_tasks(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tvm/meta_schedule/tune.py", line 122, in tune_tasks
    task_scheduler.tune(
  File "/usr/local/lib/python3.11/dist-packages/tvm/meta_schedule/task_scheduler/task_scheduler.py", line 132, in tune
    _ffi_api.TaskSchedul

After tuning, you are expected to see the table
```
 ID | Name |        FLOP | Weight | Speed (GFLOPS) | Latency (us) | Weighted Latency (us) | Trials | Done
----------------------------------------------------------------------------------------------------------
  0 | main | 17188257792 |      1 |     xxxxxxxxxx |   xxxxxxxxxx |            xxxxxxxxxx |    xxx |    Y
----------------------------------------------------------------------------------------------------------
```
in the end, and a function `apply_trace` printed out:
```python
# from tvm import tir
def apply_trace(sch: tir.Schedule) -> None:
  b0 = sch.get_block(name="gemm", func_name="main")
  # ...
```

Please, **copy the entire `apply_trace` function, and paste it into `trace_submission.py`**.
`trace_submission.py` is the only file you will submit for this part.

The column "Latency" in the table shows the execution time you get after auto tuning.
**When grading, we will manually run the command in the cell below to measure the execution time of the schedule in `trace_submission.py` on Google Colab with NVIDIA T4 GPU.**
You can also run the cell below to check the execution time.
The time it shows might be slightly longer than the latency shown in the table due to variance,
which is normal.

In [13]:
!python3 evaluate.py --evaluate-tuned

Traceback (most recent call last):
  File "/content/drive/MyDrive/15442/assignment2/evaluate.py", line 110, in <module>
    evaluate_execution_time(sch)
  File "/content/drive/MyDrive/15442/assignment2/evaluate.py", line 40, in evaluate_execution_time
    f = build_sch(sch)
        ^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/15442/assignment2/evaluate.py", line 15, in build_sch
    return tvm.build(sch.mod, target="cuda")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tvm/driver/build_module.py", line 297, in build
    rt_mod_host = _driver_ffi.tir_to_runtime(annotated_mods, target_host)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "tvm/_ffi/_cython/./packed_func.pxi", line 339, in tvm._ffi._cy3.core.PackedFuncBase.__call__
  File "tvm/_ffi/_cython/./packed_func.pxi", line 270, in tvm._ffi._cy3.core.FuncCall
  File "tvm/_ffi/_cython/./packed_func.pxi", line 259, in tvm._ffi._cy3.core.FuncCall3
  File "t

The scores you will get is based on the execution time you can reach.
Here is the score table for your reference:

| Execution time after tuning (ms) | Scores (pt) |
| -: | :-: |
| ≤ 20.00 | 20 |
| ≤ 40.00 | 15 |
| ≤ 80.00 | 10 |
| ≤ 160.00 | 5  |

## Part 3. Assignment Feedback (0 pt)

This is the second time we offer this course, and we appreciate any assignment feedback from you.
You can leave your feedback (if any) in `feedback.txt`, and submit it together with the source code.
Possible choices can be:

- Are the notebooks and instructions clear enough?
- How difficult do you think this assignment is?
- How much time does the assignment take? Which task takes the most time?
- Which part of the assignment do you feel hard to understand?
- And any other things you would like to share.

Your feedback will be very useful in helping us improve the assignment quality
for next years.


## Code Submission and Grading

In the home directory for the assignment, execute the command

In [ ]:
!zip handin.zip gemm_relu_add.py trace_submission.py feedback.txt

This will create a zip file with `gemm_relu_add.py`, `trace_submission.py` and `feedback.txt`.
You can check the contents of `handin.zip` to make sure it contains all the needed files:

In [ ]:
!zipinfo -1 handin.zip

It is expected to list the three files:
```
gemm_relu_add.py
trace_submission.py
feedback.txt
```

Then, please go to GradeScope at https://www.gradescope.com/courses/951055 and submit the file `handin.zip` to Assignment 2.

The assignment will be automatically graded.
* For both parts, the autograder checks both the optimization completeness and the numerical correctness. You will not get full scores if the autograder finds your optimization schedule is numerically incorrect.
* For `gemm_relu_add.py`, please make sure NOT to change the signatures of `shared_memory_tiling`, `register_tiling`, `cooperative_fetching`, `write_cache` and `epilogue_fusion`, or otherwise the autograder may not process your submission properly.
* Please make sure that your submitted `gemm_relu_add.py` and `trace_submission.py` are placed at the root level of the zip file (i.e., they are not in any sub-folder),
or **otherwise the autograder may not process your submission properly**.
* You can submit multiple times, and the time stamp of the last submission will be used in determining any late penalties.

**Any attempt to manipulate or compromise the integrity of the autograder will result in severe penalties.**

If you are enrolled in the course (on SIO), but not registered on GradeScope, please let the course staff know in a private post on Piazza.
